<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# [0]

import pandas as pd

# loading data from a csv fle
url1 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/corolla_1.csv'
url2 = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/corolla_2.csv'
data1 = pd.read_csv(url1)
data2 = pd.read_csv(url2)
data = pd.merge(left=data1, right=data2, on='ID')

# removing null values(records)
data = data.dropna()

# creating a list of column names
cols = list(data.columns)

In [0]:
# [1]

from scipy import stats
import math

price_auto = data[data['TransmissionType'] == 'Automatic']['Price']
price_manu = data[data['TransmissionType'] == 'Manual']['Price']

t_value, p_value = stats.ttest_ind(price_auto, price_manu)

print(f'Answer [1] : p-value = {math.floor(p_value * 1000) / 1000}')

Answer [1] : p-value = 0.095


In [0]:
# [2]

doors_2 = data[data['Doors'] == '2doors']['Price']
doors_3 = data[data['Doors'] == '3doors']['Price']
doors_4 = data[data['Doors'] == '4doors']['Price']
doors_5 = data[data['Doors'] == '5doors']['Price']

f_value, p_value = stats.f_oneway(doors_2, doors_3, doors_4, doors_5)

print(f'Answer [2] : f-value = {math.floor(f_value * 100) / 100}, ' +
      f'p-value = {math.floor(p_value * 10000) / 10000}')

Answer [2] : f-value = 6.65, p-value = 0.0001


In [0]:
# [3]

import statsmodels.api as sm

data_corr = data.corr(method='pearson')
corr_list = list(data_corr[abs(data_corr['Price']) >= 0.3].index)
corr_list.remove('ID')
corr_list.remove('Price')
data_cat = pd.get_dummies(data, columns=cols[7:], drop_first=True)

train_x = pd.concat([data[corr_list], data_cat[data_cat.columns[7:]]], axis=1)
train_y = data['Price']

train_x = sm.add_constant(train_x)
model = sm.OLS(train_y, train_x)
results = model.fit()

cnt = 0
for index, pvalue in enumerate(results.pvalues):
    if index == 0:
        continue
    if pvalue <= 0.05:
        cnt += 1

print(f'Answer [3] No of variant = {cnt}, ' +
      f'Adjusted R2 = {math.floor(results.rsquared_adj * 1000) / 1000:.3f}\n')

Answer [3] No of variant = 5, Adjusted R2 = 0.860



/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [0]:
# [4]

value = [[1.0, 5, 50000, 143, 1170, 1, 0, 0, 0, 1, 0]]
columns = ['const', 'Age', 'KM', 'HP', 'Weight', 'FuelType_Diesel',
           'FuelType_Petrol', 'TransmissionType_Manual', 'Doors_3doors',
           'Doors_4doors', 'Doors_5doors']
test = pd.DataFrame(value, columns=columns)

predicts = results.predict(test)

print(f'Answer [4] : Predicted price = {math.floor(predicts)}')

Answer [4] : Predicted price = 21053
